In [248]:
import os
import numpy as np
import pytesseract
import cv2
import matplotlib.pyplot as plt
import json

# 실행파일 위치저장
pytesseract.pytesseract.tesseract_cmd = "C:/Tesseract-OCR/tesseract.exe"

# 이미지 파일명
img_name = "raven9.jpg"

# 이미지 읽기
img = cv2.imread(img_name)

# 이미지 크기 확인
height, width, _ = img.shape

# 위치 배열 초기화 & 추가
locations = []
def add_location(name, x1, y1, x2, y2, mode):
    locations.append({'name': name,'x1': x1,'y1': y1,'x2': x2,'y2': y2,'mode': mode })

def show(template):
    plt.imshow(template)
    plt.show()

class OCR:
    def __init__(self):
        pass
 
    def get(self, img, mode):
        
        # show(img)
        # 전처리        
        if mode == 'kor':
            roi_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, roi = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
        elif mode == 'number':
            roi_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, roi = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        elif mode == 'eng':
            roi_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            roi = roi_gray

        # show(roi)
        # 이미지 스캔             
        if mode == 'kor':
            text = pytesseract.image_to_string(roi, lang='kor+eng --psm 6')
            return text
        elif mode == 'number':
            text = ''.join(filter(str.isdigit, pytesseract.image_to_string(roi, config='--psm 6')))
            return text
        elif mode == 'eng':
            text = ''.join(filter(str.isalpha, pytesseract.image_to_string(roi, config='eng --oem')))
            return text
        else:
            return "Invalid language type"
    

In [249]:

# 클래스 인스턴스 생성
ocr = OCR()

# 결과값 dict
result = {}

# 위치 정보를 추가
add_location('no', 292, 163, 330, 185, 'number')
add_location('command', 356, 168, 1250, 210, 'kor')
add_location('name', 348, 135, 604, 169, 'eng')

# 위치 배열을 순회하며 이미지에서 자르고 텍스트 읽어오기
for idx, loc in enumerate(locations):
    name, mode = loc['name'], loc['mode']
     # 이미지에서 특정 구간 자르기 
    img_roi = img[loc['y1']:loc['y2'], loc['x1']:loc['x2']]
    
    item = {}
    item['text'] = ocr.get(img_roi, mode) # OCR 실행
    # item['location'] = loc
    result[name] = item 
        
# 읽어온 텍스트와 위치 정보 출력 
print(json.dumps(result, indent=4, ensure_ascii=False))

with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)

{
    "no": {
        "text": "122"
    },
    "command": {
        "text": "상대의 공격에 맞춰서\n"
    },
    "name": {
        "text": "UtsusemiEscape"
    }
}
